#**CycleGAN Keras İmplemantasyonu**

***Bu çalışma CycleGAN ile fotoğraflarda kapalı çıkan bir gözün açık haldeki tahminlemesi. Vanilla versiyon.***

#CycleGAN Algoritması

![alt text](https://i.hizliresim.com/qCV8Y0.jpg)




#1-Veri Hazırlama  
CycleGAN doğası gereği pair fotoğraflara ihtiyaç duymaz. Fakat Modelimizin 300 milyon parametre içermesi, büyüklüğü ve eğitim zorluğu beni kendi pair çiftlerimi oluşturmaya itti. Açıkçası pek bir farkı yok gibi görünsede ve internette uzun araştırmalarım sonucunda en ufak 1 bilgi bulamasam bile bu konu hakkında hesaplama kolaylığı olduğunu düşünüyorum.


23k gözleri açık facedatasetini cv2 kütüphanesinde bulunan Cascade Classifier'ı kullanarak gözlerini tespit ettim ve beyazla boyadım.
 
Fakat ileride sadece 1k'sını kullanıcam.

(source code'a eklediğim veriseti bu işlemin en son halidir)

In [0]:
!nvidia-smi

Fri Jun 12 20:11:31 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!pip install -q kaggle

In [0]:
!mkdir ~/.kaggle

In [0]:
!cp kaggle.json ~/.kaggle/

In [0]:
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
!kaggle datasets list

In [0]:
!kaggle datasets download -d abhikjha/utk-face-cropped

In [0]:
!unzip utk-face-cropped.zip -d /content/Photos

In [0]:
cd /content/Photos/utkcropped

In [0]:
!rm -r utkcropped

In [0]:
import os
arr = os.listdir('.')
for i in range(len(arr)):
  os.rename(r'/content/Photos/utkcropped/{}'.format(arr[i]),r'/content/Photos/utkcropped/{}.jpg'.format(str(i)))
  ##dosya isimlerini 0.jpg'den 23708.jpg'ye kadar numaralandırma

In [0]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt

eye_cascade = cv2.CascadeClassifier("/content/eyedetection.xml")

In [0]:
def photoisokay(eyes):
  if len(eyes) == 2:
    return True
  else:
    return False

In [0]:
for index in range(len(arr)):

  image_real = cv2.imread(arr[index])
  image_fake = image_real.copy()

  eyes = eye_cascade.detectMultiScale(image_fake,scaleFactor=1.1,minNeighbors=10)

  if not photoisokay(eyes):
    continue

  for (x,y,w,h) in eyes:
    cv2.rectangle(image_fake , (x,y) , (x+w,y+h) , (255,255,255) , -1)


  cv2.imwrite('/content/Real/image{}.jpg'.format(index), image_real)
  cv2.imwrite('/content/Fake/image{}.jpg'.format(index), image_fake)

#CycleGAN

Modeldeki kayıplar optimizer'lar ve çeşitli parametreler orjinal CycleGAN'ın pytorch implementation'ından alınmıştır. Zira bu modeli bile eğitmek 3 gün sürdüğü için gridsearch yapma şansım yok.

In [0]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

In [0]:
from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model, Input
from keras.layers import Conv2D, LeakyReLU, Activation, Concatenate, BatchNormalization, Conv2DTranspose
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.preprocessing.image import img_to_array, load_img
from keras.utils.vis_utils import plot_model
import keras.backend as K
K.clear_session()

import numpy as np
import os
from matplotlib import pyplot

Veri İşlemleri Operasyonları

In [0]:
class DataOperations():

 
  @staticmethod #Resimlerin boyutunu 256,256 yapıp array'e dönüştürüp npz uzantılı kaydeden metod
  def import_and_save_images(name, path, size=(256,256)):  
    data_list = list()
    img_list = os.listdir(path)
    img_list.sort()

    count = 0
    for filename in img_list:
      pixels = load_img(path + filename, target_size=size)
      pixels = img_to_array(pixels)
      data_list.append(pixels)
      count += 1
      if count%1 == 0:
        print(count, ". Appended")
    
    filename = name + '_256.npz'
    np.savez_compressed(path+filename, np.asarray(data_list))
    print('Saved dataset: ', filename)
    return filename

  
  @staticmethod #kaydedilmiş resimleri [-1,1] aralığına sokup geri yükleyen metod
  def load_samples(filename1,filename2):

    file1 = np.load(filename1)
    file1 = file1['arr_0']
    file1 = (file1 - 127.5) / 127.5

    file2 = np.load(filename2)
    file2 = file2['arr_0']
    file2 = (file2 - 127.5) / 127.5

    print('Loaded: ',filename1," and " ,filename2)
    return file1,file2

CycleGAN class'ı.

In [0]:
class CycleGAN():

  @staticmethod #discriminatör modeli
  def define_discriminator(image_shape):
    init_disc = RandomNormal(stddev=0.02) #weight init

    input_image = Input(shape=image_shape)  #input

    d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init_disc)(input_image)
    d = LeakyReLU(alpha=0.2)(d)

    d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init_disc)(d)
    d = InstanceNormalization(axis=-1)(d)
    d = LeakyReLU(alpha=0.2)(d)

    d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init_disc)(d)
    d = InstanceNormalization(axis=-1)(d)
    d = LeakyReLU(alpha=0.2)(d)

    d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init_disc)(d)
    d = InstanceNormalization(axis=-1)(d)
    d = LeakyReLU(alpha=0.2)(d)

    d = Conv2D(512, (4,4), padding='same', kernel_initializer=init_disc)(d)
    d = InstanceNormalization(axis=-1)(d)
    d = LeakyReLU(alpha=0.2)(d)

    output_image = Conv2D(1, (4,4), padding='same', kernel_initializer=init_disc)(d)

    model = Model(input_image, output_image)
    model.compile(loss='mse', optimizer=Adam(lr=0.0002, beta_1=0.5), loss_weights=[0.5])
    return model

  @staticmethod #resnet block of generator
  def resnet_block(n_filters, input_layer):

    init = RandomNormal(stddev=0.02)

    g = Conv2D(n_filters, (3,3), padding='same', kernel_initializer=init)(input_layer)
    g = InstanceNormalization(axis=-1)(g)
    g = Activation('relu')(g)
  
    g = Conv2D(n_filters, (3,3), padding='same', kernel_initializer=init)(g)
    g = InstanceNormalization(axis=-1)(g)

    g = Concatenate()([g, input_layer])
    return g

  @staticmethod #generator model
  def define_generator(image_shape=(256,256,3), n_resnet=9):

    init = RandomNormal(stddev=0.02)

    in_image = Input(shape=image_shape)

    g = Conv2D(64, (7,7), padding='same', kernel_initializer=init)(in_image)
    g = InstanceNormalization(axis=-1)(g)
    g = Activation('relu')(g)

    g = Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
    g = InstanceNormalization(axis=-1)(g)
    g = Activation('relu')(g)

    g = Conv2D(256, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
    g = InstanceNormalization(axis=-1)(g)
    g = Activation('relu')(g)

    for _ in range(n_resnet):
      g = CycleGAN.resnet_block(256, g)

    g = Conv2DTranspose(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
    g = InstanceNormalization(axis=-1)(g)
    g = Activation('relu')(g)

    g = Conv2DTranspose(64, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
    g = InstanceNormalization(axis=-1)(g)
    g = Activation('relu')(g)

    g = Conv2D(3, (7,7), padding='same', kernel_initializer=init)(g)
    g = InstanceNormalization(axis=-1)(g)
    out_image = Activation('tanh')(g)

    model = Model(in_image, out_image)
    return model

  @staticmethod #Komposit model
  def define_composite_model(g_model_1, d_model, g_model_2, image_shape):

    g_model_1.trainable = True
    d_model.trainable = False
    g_model_2.trainable = False

    # Adverisal Model
    input_gen = Input(shape=image_shape)
    gen1_out = g_model_1(input_gen)
    output_d = d_model(gen1_out)

    # Identity Model
    input_id = Input(shape=image_shape)
    output_id = g_model_1(input_id)

    # Forward Model
    output_f = g_model_2(gen1_out)

    # Backward cycle
    gen2_out = g_model_2(input_id)
    output_b = g_model_1(gen2_out)

    model = Model([input_gen, input_id], [output_d, output_id, output_f, output_b])
    model.compile(loss=['mse', 'mae', 'mae', 'mae'], loss_weights=[1, 5, 10, 10], optimizer=Adam(lr=0.0002, beta_1=0.5))
    return model

  @staticmethod #Veri setinden index'ten gelen fotoğrafı çekmek ve discriminator'e öğretmek
  def generate_real_samples(dataset, index=0 ,n_samples=1, patch_shape=16):

    X = dataset[[index]]
    y = np.ones((n_samples, patch_shape, patch_shape, 1))
    return X, y
  
  @staticmethod #generator modelden fake foto üretmek ve discriminator'e öğretmek
  def generate_fake_samples(g_model, dataset, patch_shape=16):

    X = g_model.predict(dataset)
    y = np.zeros((len(X), patch_shape, patch_shape, 1))
    return X, y

  @staticmethod #Modelin karmaşıklığını arttırmak için pool'a fotoğraf ekleyip rastgele seçim yapmak.
  def update_image_pool(pool, images, max_size=50):
    selected = list()
    for image in images:
      
      if len(pool) < max_size:
        pool.append(image)
        selected.append(image)
      
      elif random() < 0.5:
        selected.append(image)
      
      else:
        ix = randint(0, len(pool))
        selected.append(pool[ix])
        pool[ix] = image

    return np.asarray(selected)

  @staticmethod #Modeli ve ağırlıklarını savelemek
  def save_models(step, g_model_AtoB, g_model_BtoA , d_model_A, d_model_B):
    # save the first generator model
    filename1 = '/content/drive/My Drive/image1k/g_model_AtoB_%06d.h5' % (step+1)
    g_model_AtoB.save(filename1)
    # save the second generator model
    filename2 = '/content/drive/My Drive/image1k/g_model_BtoA_%06d.h5' % (step+1)
    g_model_BtoA.save(filename2)

    filename3 = '/content/drive/My Drive/image1k/d_model_A_%06d.h5' % (step+1)
    d_model_A.save(filename3)

    filename4 = '/content/drive/My Drive/image1k/d_model_B_%06d.h5' % (step+1)
    d_model_B.save(filename4)

    print('>Saved: %s and %s and %s and %s' % (filename1, filename2, filename3, filename4))

  @staticmethod #Fotoğraf özeti için aslından farklı y üretmeyen fonksiyon
  def generate_real_samples_for_summary(dataset, n_samples, patch_shape):

    ix = np.random.randint(0, dataset.shape[0], n_samples)
    X = dataset[ix]
    y = np.ones((n_samples, patch_shape, patch_shape, 1))
    return X, y

  @staticmethod #performans özet
  def summarize_performance(step, g_model, trainX, name, n_samples=5):

    X_in, _ = CycleGAN.generate_real_samples_for_summary(trainX, n_samples, 0)

    X_out, _ = CycleGAN.generate_fake_samples(g_model, X_in, 0)

    X_in = (X_in + 1) / 2.0
    X_out = (X_out + 1) / 2.0

    for i in range(n_samples):
      pyplot.subplot(2, n_samples, 1 + i)
      pyplot.axis('off')
      pyplot.imshow(X_in[i])

    for i in range(n_samples):
      pyplot.subplot(2, n_samples, 1 + n_samples + i)
      pyplot.axis('off')
      pyplot.imshow(X_out[i])

    filename1 = '/content/drive/My Drive/image1k/%s_generated_plot_%06d.png' % (name, (step+1))
    pyplot.savefig(filename1)
    pyplot.close()

  @staticmethod #train kısmı
  def train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, realdata, nrrdata):
    
    n_epochs, n_batch, = 100, 1
    n_patch = d_model_A.output_shape[1]
    
    trainA, trainB = realdata, nrrdata
    poolA, poolB = list(), list()

    n_steps = int(len(trainA)) * n_epochs


    for i in range(n_steps):

      #index = np.random.randint(0,1000)  #pair üretimi için
      #X_realA, y_realA = CycleGAN.generate_real_samples(trainA,index, n_batch, n_patch)
      #X_realB, y_realB = CycleGAN.generate_real_samples(trainB,index, n_batch, n_patch)

      X_realA, y_realA = CycleGAN.generate_real_samples_for_summary(trainA, n_batch, n_patch)
      X_realB, y_realB = CycleGAN.generate_real_samples_for_summary(trainB, n_batch, n_patch)


      X_fakeA, y_fakeA = CycleGAN.generate_fake_samples(g_model_BtoA, X_realB, n_patch)
      X_fakeB, y_fakeB = CycleGAN.generate_fake_samples(g_model_AtoB, X_realA, n_patch)

      #X_fakeA = CycleGAN.update_image_pool(poolA, X_fakeA)
      #X_fakeB = CycleGAN.update_image_pool(poolB, X_fakeB)


      g_loss2, _, _, _, _  = c_model_BtoA.train_on_batch([X_realB, X_realA], [y_realA, X_realA, X_realB, X_realA])


      dA_loss1 = d_model_A.train_on_batch(X_realA, y_realA)
      dA_loss2 = d_model_A.train_on_batch(X_fakeA, y_fakeA)


      g_loss1, _, _, _, _ = c_model_AtoB.train_on_batch([X_realA, X_realB], [y_realB, X_realB, X_realA, X_realB])


      dB_loss1 = d_model_B.train_on_batch(X_realB, y_realB)
      dB_loss2 = d_model_B.train_on_batch(X_fakeB, y_fakeB)
  

      print('>%d, dA[%.3f,%.3f] dB[%.3f,%.3f] g[%.3f,%.3f]' % (i+1, dA_loss1,dA_loss2, dB_loss1,dB_loss2, g_loss1,g_loss2))



      if (i+1) % int(len(trainA)) == 0:
        CycleGAN.summarize_performance(i, g_model_AtoB, trainA, 'AtoB')
        CycleGAN.summarize_performance(i, g_model_BtoA, trainB, 'BtoA')

        CycleGAN.save_models(i, g_model_AtoB, g_model_BtoA , d_model_A, d_model_B)

        

#ANA OPERASYONLAR

200x200 veriyi 256x256 ya dönüştürüp .npz olarak kaydettiğimiz kısım

In [0]:
real_path = "/content/drive/My Drive/image1k/Real1k/"
nrr_path = "/content/drive/My Drive/image1k/NotReallyReal1k/"

realName = DataOperations.import_and_save_images("real",real_path,size=(256,256))
nrrName = DataOperations.import_and_save_images("nrr",nrr_path,size=(256,256))

.npz dosyasından import etmek

In [0]:
path_1k = "/content/drive/My Drive/image1k/"
realdata , nrrdata = DataOperations.load_samples(path_1k+"real_256.npz", path_1k+"nrr_256.npz")

Loaded:  /content/drive/My Drive/image1k/real_256.npz  and  /content/drive/My Drive/image1k/nrr_256.npz


İlk çalıştırma için modellerin oluşturulması

In [0]:
image_shape = realdata.shape[1:]

g_model_AtoB = CycleGAN.define_generator(image_shape)
g_model_BtoA = CycleGAN.define_generator(image_shape)

d_model_A = CycleGAN.define_discriminator(image_shape)
d_model_B = CycleGAN.define_discriminator(image_shape)

c_model_AtoB = CycleGAN.define_composite_model(g_model_AtoB, d_model_B, g_model_BtoA, image_shape)
c_model_BtoA = CycleGAN.define_composite_model(g_model_BtoA, d_model_A, g_model_AtoB, image_shape)

Daha önce çalıştırılmışsa modeli load edip çalıştırmak. Ya üstteki ya alttaki kod çalışır.

In [0]:
import keras.backend as K
K.clear_session()


from keras.models import load_model
cust = {'InstanceNormalization': InstanceNormalization}

g_model_AtoB = load_model('/content/drive/My Drive/image1k/g_model_AtoB_002500.h5', cust, compile=False)
g_model_BtoA = load_model('/content/drive/My Drive/image1k/g_model_BtoA_002500.h5', cust, compile=False)

d_model_A = load_model('/content/drive/My Drive/image1k/d_model_A_002500.h5', cust, compile=True)

d_model_B = load_model('/content/drive/My Drive/image1k/d_model_B_002500.h5', cust, compile=True)


image_shape = realdata.shape[1:]
c_model_AtoB = CycleGAN.define_composite_model(g_model_AtoB, d_model_B, g_model_BtoA, image_shape)
c_model_BtoA = CycleGAN.define_composite_model(g_model_BtoA, d_model_A, g_model_AtoB, image_shape)

Train başlasın

In [0]:
if __name__ == "__main__":
  CycleGAN.train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, realdata, nrrdata)

In [0]:
#75.5k train 75.5 epoch
sample = "/content/77.jpg"
sample = load_img(sample, target_size=(256,256))
sample_array = img_to_array(sample)
listx = []
listx.append(sample_array)
sample_array = np.asarray(listx)
sample_array = (sample_array-127.5)/127.5
sonuc = g_model_BtoA.predict(sample_array)
sonuc = (sonuc + 1) /2
pyplot.imshow(sonuc[0])

#Sonuç


![alt text](https://github.com/nailcankara/DL/blob/master/07_Generative_Adversarial_Networks(CycleGAN)/22.jpg?raw=true)

![alt text](https://raw.githubusercontent.com/nailcankara/DL/master/07_Generative_Adversarial_Networks(CycleGAN)/22new1.jpg)

*On translation tasks that involve color and texture changes, like many of those reported above, the method often succeeds. We have also explored tasks that require geometric changes, with little success.*

Cycle Gan renk ve doku üretimi/değişimi konusunda başarılır. Geometrik şekillerde ise oldukça başarısız